# Normalize CSV columns

Normalize the columns for the eODP CSVs.

In [1]:
import sys
sys.path.append('../scripts/')

import pandas as pd

from normalize_data import (
    normalize_sample_col, 
    normalize_expedition_section_cols, 
    csv_cleanup,
    update_metadata,
    fetch_unique_column_names,
    append_set,
    filter_existing_set,
    normalize_columns,
    add_missing_columns
)

In [2]:
lithology = 'cleaned_data/Lithology_CSV'
lithology_meta = 'cleaned_data/metadata/Lithology_changes.csv'

micropal_1 = 'cleaned_data/Micropal_CSV_1'
micropal_meta_1 = 'cleaned_data/metadata/Micropal_1_changes.csv'

micropal_2 = 'cleaned_data/Micropal_CSV_2'
micropal_meta_2 = 'cleaned_data/metadata/Micropal_2_changes.csv'

micropal_3 = 'cleaned_data/Micropal_CSV_3'
micropal_meta_3 = 'cleaned_data/metadata/Micropal_3_changes.csv'

In [3]:
metadata_file = micropal_meta_3
clean_data_path = micropal_3

## Normalize expedition..section columns 

Read each Lithology CSV to check if expedition..section columns exist. Overwrite existing Lithology CSV if columns need to be added.

### Read metadata file

In [4]:
metadata = pd.read_csv(metadata_file)
metadata.head()

,file,taxon_group
0,339_benthic_forams_U1388B_5.csv,benthic_forams
1,324_U1348A_benthic_forams.csv,benthic_forams
2,339_planktic_forams_U1387C.csv,planktic_forams
3,339_benthic_forams_U1390A_6.csv,benthic_forams
4,341_radiolarians_U1419D.csv,radiolarians


In [5]:
metadata.shape

(416, 2)

### Normalize columns

In [6]:
def process_filename(file):
    path = f"{clean_data_path}/{file}"
    content = pd.read_csv(path)
    
    original_cols = content.columns
    content = normalize_expedition_section_cols(content)
    changed = list(original_cols) != list(content.columns)

        
    if changed:
        content = csv_cleanup(content, path)
        content.to_csv(path, index=False)

    return changed
    
change_columns = [process_filename(file) for file in metadata['file']] 


### Update metadata

In [7]:
dict = {"add_expedition_section_cols": change_columns}
new_metadata = update_metadata(metadata, dict)
new_metadata.head()

,file,taxon_group,add_expedition_section_cols
0,339_benthic_forams_U1388B_5.csv,benthic_forams,False
1,324_U1348A_benthic_forams.csv,benthic_forams,False
2,339_planktic_forams_U1387C.csv,planktic_forams,False
3,339_benthic_forams_U1390A_6.csv,benthic_forams,False
4,341_radiolarians_U1419D.csv,radiolarians,True


In [8]:
new_metadata.to_csv(metadata_file, index=False)

## Normalize Sample column

Read each Lithology CSV to check if Sample column needs to be updated. Change 'Label ID' to 'Sample'. Add 'Sample' if no 'Sample' or 'Label ID' based on expedition...section columns. Overwrite existing Lithology CSV if Sample column is updated.

### Read metadata file

In [9]:
metadata = pd.read_csv(metadata_file)

In [10]:
metadata.shape

(416, 3)

### Normalize columns

In [11]:
def process_filename(file):
    path = f"{clean_data_path}/{file}"
    content = pd.read_csv(path)
    
    original_cols = content.columns
    normalize_sample_col(content)
    
    changed = list(original_cols) != list(content.columns)
    
    if changed:
        content = csv_cleanup(content, path)
        content.to_csv(path, index=False)

    return changed
    
change_columns = [process_filename(file) for file in metadata['file']] 

### Update metadata

In [12]:
dict = {"update_sample_col": change_columns}
new_metadata = update_metadata(metadata, dict)
new_metadata.head()

,file,taxon_group,add_expedition_section_cols,update_sample_col
0,339_benthic_forams_U1388B_5.csv,benthic_forams,False,False
1,324_U1348A_benthic_forams.csv,benthic_forams,False,True
2,339_planktic_forams_U1387C.csv,planktic_forams,False,False
3,339_benthic_forams_U1390A_6.csv,benthic_forams,False,False
4,341_radiolarians_U1419D.csv,radiolarians,True,False


In [13]:
new_metadata.to_csv(metadata_file, index=False)

## Normalize Top and Bottom columns

Normalize all the Top, Top Depth, Bottom, and Bottom Depth column to have the same names.

In [14]:
metadata = pd.read_csv(metadata_file)

### Get top bottom columns

In [15]:
columns_all = set()

res=[fetch_unique_column_names(f"{clean_data_path}/{file}", columns_all) for file in metadata['file']] 

In [16]:
top_all = set()
top = set()
top_depth = set()

bottom_all = set()
bottom = set()
bottom_depth = set()

append_set(top_all, r".*?top.*?", columns_all)
append_set(top_depth, r"top depth", columns_all)
append_set(top, r"top offset|top ?\[", columns_all)

append_set(bottom_all, r".*?bottom.*?", columns_all)
append_set(bottom_depth, r"bottom depth", columns_all)
append_set(bottom, r"bottom offset|bottom ?\[", columns_all)

In [17]:
top_all

{'Acarinina praetopilensis ',
 'Acarinina pseudotopilensis',
 'Acarinina topilensis',
 'Acrocubus octopylus',
 'Anogramma leptophylla',
 'Artophormis barbadensis',
 'Artophormis dominasinensis',
 'Artophormis gracilis',
 'Calcidiscus leptoporus',
 'Calcidiscus leptoporus ',
 'Calcidiscus leptoporus (5-8 microns)',
 'Calcidiscus leptoporus (<5 microns)',
 'Calcidiscus leptoporus (>8 micron)',
 'Calcidiscus leptoporus (small 3-5 um)',
 'Cushmanina striatopunctata',
 'Eprolithus octopetalus',
 'Grammatophora spp.',
 'Octopyle stenozona',
 'Octopyle/Tetrapyle Group',
 'Pylospira octopyle',
 'Thalassiosira leptopus',
 'Thalssiosira leptopus',
 'Top Depth [m]',
 'Top Depth[m] [m]',
 'Top [cm]',
 'Top depth [m]',
 'Top[cm] [cm]',
 'Zygodiscus plectopons'}

In [18]:
top

{'Top [cm]', 'Top[cm] [cm]'}

In [19]:
top_depth

{'Top Depth [m]', 'Top Depth[m] [m]', 'Top depth [m]'}

In [20]:
bottom_all

{'Bottom Depth [m]',
 'Bottom Depth[m] [m]',
 'Bottom [cm]',
 'Bottom depth [m]',
 'Bottom[cm] [cm]'}

In [21]:
bottom

{'Bottom [cm]', 'Bottom[cm] [cm]'}

In [22]:
bottom_depth

{'Bottom Depth [m]', 'Bottom Depth[m] [m]', 'Bottom depth [m]'}

### Normalize top bottom columns

In [23]:
def normalize_top_bottom(file):
    path = f"{clean_data_path}/{file}"
    content = pd.read_csv(path)
    columns = list(content.columns)
    
    normalized_cols = normalize_columns(top, 'Top [cm]', columns)
    normalized_cols = normalize_columns(bottom, 'Bottom [cm]', normalized_cols)
    normalized_cols = normalize_columns(top_depth, 'Top Depth [m]', normalized_cols)
    normalized_cols = normalize_columns(bottom_depth, 'Bottom Depth [m]', normalized_cols)
    
    changed = columns != normalized_cols
    
    if changed:
        content.columns = normalized_cols
        content = csv_cleanup(content, path)
        content.to_csv(path, index=False)

    return changed

change_columns = [normalize_top_bottom(file) for file in metadata['file']] 

### Update metadata

In [24]:
dict = {"update_top_bottom": change_columns}
new_metadata = update_metadata(metadata, dict)
new_metadata.head()

,file,taxon_group,add_expedition_section_cols,update_sample_col,update_top_bottom
0,339_benthic_forams_U1388B_5.csv,benthic_forams,False,False,True
1,324_U1348A_benthic_forams.csv,benthic_forams,False,True,True
2,339_planktic_forams_U1387C.csv,planktic_forams,False,False,True
3,339_benthic_forams_U1390A_6.csv,benthic_forams,False,False,True
4,341_radiolarians_U1419D.csv,radiolarians,True,False,False


In [25]:
new_metadata.to_csv(metadata_file, index=False)

## Add missing columns

In [26]:
metadata = pd.read_csv(metadata_file)

In [27]:
normalized_columns = [
    'Top [cm]',
    'Bottom [cm]',
    'Top Depth [m]',
    'Bottom Depth [m]', 
    'Sample',
    'Exp',
    'Site',
    'Hole',
    'Core',
    'Type',
    'Section',
    'A/W'
]

In [28]:
change_columns = [add_missing_columns(f"{clean_data_path}/{file}", normalized_columns) for file in metadata['file']] 

In [29]:
dict = {"add_missing_cols": change_columns}
new_metadata = update_metadata(metadata, dict)
new_metadata.head()

,file,taxon_group,add_expedition_section_cols,update_sample_col,update_top_bottom,add_missing_cols
0,339_benthic_forams_U1388B_5.csv,benthic_forams,False,False,True,False
1,324_U1348A_benthic_forams.csv,benthic_forams,False,True,True,False
2,339_planktic_forams_U1387C.csv,planktic_forams,False,False,True,False
3,339_benthic_forams_U1390A_6.csv,benthic_forams,False,False,True,False
4,341_radiolarians_U1419D.csv,radiolarians,True,False,False,False


In [30]:
new_metadata.to_csv(metadata_file, index=False)